In [1]:
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, SeparableConv2D
from keras.layers import RepeatVector, TimeDistributed
from keras.layers import concatenate, add, Lambda
from keras.layers import LSTM
from keras.models import Model
from keras.regularizers import l2

from keras.applications import imagenet_utils, Xception
from keras.preprocessing.image import img_to_array
import keras.backend as K

import numpy as np 
import json
import os
import cv2
import h5py
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [3]:
from helpers import TrainingMonitor
from helpers import Utils

In [4]:
Xoh_file = open('../input/dataset/Xoh.pickle', 'rb')
x = pickle.load(Xoh_file)
Xoh_file.close()

In [5]:
Yoh_file = open('../input/dataset/Yoh.pickle', 'rb')
y = pickle.load(Yoh_file)
Yoh_file.close()

In [6]:
x.shape

(1000, 30, 37)

In [7]:
y[0].shape

(10, 11)

In [8]:
train_split, val_split = 0.7, 0.15

train_len = int(train_split * x.shape[0])
train_val_len = int((train_split + val_split) * x.shape[0])

x_train = x[:train_len]
x_val = x[train_len:train_val_len]
x_test = x[train_val_len:]

y_train = y[:train_len]
y_val = y[train_len:train_val_len]
y_test = y[train_val_len:]

In [9]:
x_train.shape

(700, 30, 37)

In [10]:
y_train.shape

(700, 10, 11)

In [14]:
model = Sequential() 
model.add(LSTM(128, input_shape=(30, 37)))
model.add(RepeatVector(10))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(11, activation="softmax"))) 
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"]) 
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 128)               84992     
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 10, 11)            715       
Total params: 135,115
Trainable params: 135,115
Non-trainable params: 0
_________________________________________________________________
None


In [15]:
output_path = "../output/"

print(os.getpid())
filepath=output_path + "progress/seq2seq-stacked-weights-best.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

7144


In [16]:
history = model.fit(x_train, y_train,
            batch_size=32,
            epochs=100,
            validation_data=(x_val, y_val),
            shuffle="batch",
            callbacks=callbacks)

Train on 700 samples, validate on 150 samples
Epoch 1/100
700/700 [==============================] - 3s 4ms/step - loss: 2.2298 - acc: 0.1923 - val_loss: 2.1471 - val_acc: 0.2000
Epoch 2/100
700/700 [==============================] - 1s 2ms/step - loss: 2.1150 - acc: 0.2299 - val_loss: 2.0631 - val_acc: 0.3033
Epoch 3/100
700/700 [==============================] - 1s 2ms/step - loss: 2.0098 - acc: 0.2809 - val_loss: 1.9409 - val_acc: 0.3707
Epoch 4/100
700/700 [==============================] - 1s 2ms/step - loss: 1.9150 - acc: 0.3736 - val_loss: 1.8719 - val_acc: 0.3980
Epoch 5/100
700/700 [==============================] - 1s 2ms/step - loss: 1.8428 - acc: 0.3786 - val_loss: 1.7989 - val_acc: 0.4113
Epoch 6/100
700/700 [==============================] - 1s 2ms/step - loss: 1.7752 - acc: 0.3891 - val_loss: 1.7261 - val_acc: 0.4013
Epoch 7/100
700/700 [==============================] - 1s 2ms/step - loss: 1.6943 - acc: 0.4117 - val_loss: 1.6416 - val_acc: 0.4007
Epoch 8/100
700/700 [==

In [17]:
scores = model.evaluate(x_test, y_test, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 99.70%; Val: 88.80%; Test: 88.07%


In [ ]:
model_json = model.to_json()
with open("../output/saved/date_model_seq2seq_94.33.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("../output/saved/date_weight_seq2seq_94.33.hdf5")